# Hi!
Hello and welcome in this tutorial for polytope reconfiguration of music.

This work presents a geometrical view for analyzing music, allowing to consider relations between musical elements which don't follow the sequential order.

It was developped by C. Guichaoua [1] and C. Louboutin [2], both under the supervision of F. Bimbot.

[TODO: Add My Own Reference]: In the near future, I will upload a reference (probably on ArXiv) summing up both of their work, detailing their similarities, and presenting some new models, which could help understanding the rest of this notebook. In the meantime, you should refer to the PhD thesis previously referenced.

[1] C. Guichaoua, Modèles de compression et critères de complexité pour la description et l’inférence de structure musicale.  PhD thesis, 2017.

[2]  C. Louboutin, Modélisation multi-échelle et multi-dimensionnelle de la structure musicale par graphes polytopiques. PhD thesis, Rennes 1, 2019.

# Installation
To install the code, you should donwload the project: https://gitlab.inria.fr/amarmore/musiconpolytopes

Then you should put all this in a folder, at a given path (noted /path/to/package).

Finally, you should open a console (on Linux distribution) or an anaconda prompt (on Windows) and type: `pip install -e /path/to/package`.

# From Polytopes to Patterns
In this work, polytopes are geometrical objects (n-dimensional hypercubes with possible alteration), which support musical elements on their vertices, and link them by edges. In general (at least for now), musical elements are major or minor chords, discretized on a temporal index (beats or upbeats typically). An example of polytope is presented on the figure below.

<img src="imgs/polytope_example.png" width="700"/>

To represent them computationally, we used nested lists. Indeed, polytopes are n dimensional hypercubes (which can then be altered). In that sense, a n-dimensional hypercube is constructed recursively as the union of two (n-1)-dimensional hypercubes (for instance, a cube can be seen as the concatenation of 2 squares).

Hence, in our model, a dimension is represented by the level of nesting of lists, and each vertex of the polytope will be represented by a number (for the general shape) or a chord (for the musical polytope). We call this object **pattern**, to differentiate it with the geometrical polytope.

The code used for generating and handling patterns is in the file ``pattern_factory.py``.

In [1]:
from polytopes import pattern_factory as pf

We define two types of patterns: 
- "patterns of ones", which act as the skeleton of the polytope: every vertex/element in the nested lists is 1, and represent a generic element. It is useful for considering the pattern as a whole, when differentiating elements isn't necessary (for example, evaluating each nested pattern for its shape and not its content).
    - The dimension 1 pattern of ones is [1,1],
    - The dimension 2 pattern of ones is the nesting of 2 dim-1 patterns, so [[1,1],[1,1]],
    - The dimension 3 pattern of ones is [[[1,1],[1,1]],[[1,1],[1,1]]],
    - etc.
 
- "indexed patterns", where each element of the pattern represent the index of this vertex in the chronological/sequential order. Indeed, as vertices in the polytope represent elements in a chord progression, we have to know each element's position, to relate it with the musical context. Traditionnally, indexes start at 0.
    - The dimension 1 indexed pattern is [0,1],
    - The dimension 2 indexed pattern is the nesting of 2 dim-1 patterns, so [[0,1],[2,3]],
    - The dimension 3 indexed pattern is [[[0,1],[2,3]],[[4,5],[6,7]]],
    - etc.

<img src="imgs/patterns_and_polytopes.gif" width="700"/>

In the code, generating a dim-3 pattern of ones can be made with the function `make_regular_polytope_pattern`, as presented below:

In [2]:
# Feel free to play with the dimension
pf.make_regular_polytope_pattern(dimension = 3)

[[[1, 1], [1, 1]], [[1, 1], [1, 1]]]

## Irregular polytopes and patterns
In the previous part, we presented regular polytopes and patterns, *i.e.* n-dimensional hypercubes. In our model, they represent the core of polytopes, but not the entire story. Indeed, as introduced by C. Guichaoua in [1], polytopes are extended by adding and/or deleting some vertices to a n-dimensional hypercubes. The alteration (both addition and deletion) follow the vertices of a m-dimensional polytope, with $m < n - 1$. In that sense, every polytope is a n-dimensional polytope on which some vertices may be deleted, and some other may be added, following the shape of another hypercube. In the case of both addition and deletion, the dimensions of their respective polytopic shape can be of different dimensions. When some vertices are involved with both an addition and a deletion, the deletion has higher priority.

To represent a deletion in a pattern, we simply delete the involved elements. For example, the pattern associated with the deletion of an element in a dimension 1 polytope is "[1]" (not to confuse with references), and the deletion of the last dim-1 polytope in a dim-3 polytope would be: [[[1,1],[1,1]],[[1,1]]]. (NB: a void dim-1 polytope ([]) will not be displayed).

An addition will be represented by a tuple at the position of the addition, where the first element of the tuple will be the element originally present in the polytope, and the second one will be the added element. In that sense, the pattern associated with the addition of an element in a dimension 1 polytope is "[1, (1,1)]", and the addition of a dim-1 polytope in a dim-3 polytope would be: [[[1,1],[1,1]],[[1,1],[(1,1),(1,1)]]].

### Irregularities in practice
In practice, we need a procedure to construct the irregularities. This is made by encoding the location of additions or deletions in the pattern. This encoding is in fact two lists of booleans (0 or 1) which we call codes.

Codes are constructed to indicate where and how many positions need to be altered. Even though each element in the polytope can be specified by dichotomy (dim-n polytope are recursively constructed as the concatenation of two dim-(n-1) polytopes, see part "Antecedents"), **the goal of codes here is not to indicate individually each position which need to be altered**. They rather follow a procedure, defined below:

- Codes are related to the notion of dimensions in the nesting:
     - The first element of the code (left element, the first in the list) will encode information about the 2 polytopes of dimension n-1
     - The second element of the code will encode information about polytopes at the n-2 dimension
     - The third element of the code will encode information about polytopes at the n-3 dimension
     - And so on until the last one, which represent information at the last nesting dimension (so directly on elements)
     
- At each dimension, the boolean indicates the propagation of the alteration, as a binary rule:
     - If the current boolean is a 1, this alteration will affect both nested polytopes.
     - If the current boolean is a 0, this alteration will only affect the second nested polytope, the one "on the right" (geometrically).

     In that sense, at every dimension, if the current boolean is 0, the code will only be propagated to the 2nd nested polytope, and the 1st nested polytope will be left without alteration. Otherwise, if it's a 1, the rest of the code will be copied to both polytopes of lower dimension.


- At the last level, a 0 will indicate to alter only the 2nd polytope (so the element on the right), and a 1 will indicate to alter both, as presented before.
> A consequence of that rule is that a code composed of zeroes ([0,0,...,0]) will still alter the last element! To specify "no alteration at all", code must be an empty list.

Finally, we specify, for each dimension, to which part of the nesting alteration should be propagated (both or only the second part). Hence, codes are of the same length as the dimension of the polytope.

In addition, the number of "1" in the code is exactly the dimension of the alteration polytope.

#### Illustration on a dim-3 polytope

Let's illustrate this procedure with an example: a dim-3 polytope with code [0,1,0].

<img src="imgs/code_first_presentation.png" width="500"/>

At this level, the boolean being a 0, we will propagate the alteration code to the 2nd dim-(n-1) polytope *only*, so the first dim-(n-1) polytope will remain unaltered.

<img src="imgs/code_2.png" width="700"/>

Hence, the only polytope to alter will be the polytope $(a_4, a_5, a_6, a_7)$, with the code [1,0] (the remaining part of the original code, the first boolean (0) being used at the previous level). It is a recursive process.

<img src="imgs/code_second_level.png" width="500"/>

The boolean being 1 at this level, we will propagate alteration in both nested polytopes, as presented below:

<img src="imgs/code_4.png" width="700"/>

This will propagate the remaining part of the code ([0]) on both polytopes, as presented below:

<img src="imgs/code_last_level.png" width="500"/>

In this case, the boolean 0 indicates to only alter the second dim-0 elements, but on both dim-1 polytopes ($(a_4, a_5)$ and $(a_6, a_7)$) as the code was propagated to both of them in the previous stage.

<img src="imgs/code_6.png" width="700"/>

Hence, the code [0,1,0] on a dim-3 polytope will alter elements $a_5$ and $a_7$.

Finally, an important point is that codes remain the same for addition and deletion, as they only impact position. Then, depending on the chosen alteration, the previous code can result in two polytopes, presented below:

<img src="imgs/code_7.png" width="700"/>

NB: elements in green are elements with new indexes, as addition/deletion changes their order.

#### Some examples, along with the code
Let's play with codes a little bit. For example, let's consider a polytope of dimension 3, with an addition and no deletion. The additional polytope is of dimension 1 (so 2 elements, and a unique 1 in the code), and should occur on the last elements of both nested dim-2 polytopes. It corresponds to the following polytope:

<img src="imgs/dim_3_add_100_del_.png" width="200"/>

To represent the addition, we shall propagate addition in both dim-2 polytopes, but in no other level. The code is hence a 1 for the dimension 2 level of nesting (so the first boolean element), and 0 for all the others, so: [1,0,0].

As there is no deletion, the code should be null, so the deleting code will be an empty list: [].

In [3]:
pf.make_polytope_pattern(dimension = 3, adding_code = [1,0,0], deleting_code = [])

[[[1, 1], [1, (1, 1)]], [[1, 1], [1, (1, 1)]]]

To construct the associated indexed pattern, one could use the following function:

In [4]:
pf.make_indexed_pattern(dimension = 3, adding_code = [1,0,0], deleting_code = [])

[[[0, 1], [2, (3, 4)]], [[5, 6], [7, (8, 9)]]]

Let's now consider the polytope of dimension 3, where the last 2 elements are deleted. The alteration polytope is of dimension 1 (so 2 elements, and a unique 1 in the code), and should occur on the last elements of both nested dim-0 polytopes (last two elements). It corresponds to the following polytope:

<img src="imgs/dim_3_add_del_001.png" width="200"/>

To represent the deletion, we shall only propagate addition in the last dim-0 polytopes, but in no other level. The code is hence a 1 for the dimension 0 level of nesting (so the last boolean element), and 0 for all the others, so: [0,0,1].

As there is no addition, the code should be null, so the addition code will be an empty list: [].

In [5]:
#pf.make_polytope_pattern(dimension = 3, adding_code = [], deleting_code = [0,0,1])
pf.make_indexed_pattern(dimension = 3, adding_code = [], deleting_code = [0,0,1])

[[[0, 1], [2, 3]], [[4, 5]]]

Mixing both previous codes result in a polytope where the first dim-2 polytope has an addition, and the second has its last two elements deleted (because deletion has higher priority than addition, by construction). It corresponds to the following polytope:

<img src="imgs/dim_3_add_100_del_001.png" width="200"/>

In [6]:
#pf.make_polytope_pattern(dimension = 3, adding_code = [1,0,0], deleting_code = [0,0,1])
pf.make_indexed_pattern(dimension = 3, adding_code = [1,0,0], deleting_code = [0,0,1])

[[[0, 1], [2, (3, 4)]], [[5, 6]]]

Finally, an example of a dim-4 polytope, with a dimension 2 deletion and no addition is given below: 

<img src="imgs/dim_4_add_del_0110.png" width="600"/>

To what codes does it corresponds?

In [7]:
adding_code = [] ### Change values here if needed
deleting_code = [] ### Change values here if needed
pf.make_indexed_pattern(dimension = 4, adding_code = adding_code, deleting_code = deleting_code)
# Should result in: [[[[0, 1], [2, 3]], [[4, 5], [6, 7]]], [[[8], [9]], [[10], [11]]]]

[[[[0, 1], [2, 3]], [[4, 5], [6, 7]]],
 [[[8, 9], [10, 11]], [[12, 13], [14, 15]]]]

### Codes associated to a pattern size
In practice, we may want to generate patterns according to the size of the current segment. In that sense, we want to anticipate the size of a pattern without constructing it. This is possible because alteration are of size $2^m$ with $m$ the number of 1 in the code. Hence, using the following function, we will anticipate the size of a pattern prior to its construction:

In [8]:
pf.get_final_pattern_size(dimension = 5, adding_code = [1,0,0,1,0], deleting_code = [0,1,0,1,0])

30

Thanks to that anticipation, we can also find all couple of codes which will result in an properly sized pattern:

In [9]:
# The first element of each tuple is the addition code, the second is the deletion one.
pf.get_codes(7)

[([], [0, 0, 0]),
 ([0, 0, 0], [0, 0, 0]),
 ([0, 0, 1], [0, 1, 0]),
 ([0, 0, 1], [1, 0, 0]),
 ([0, 1, 0], [0, 0, 1]),
 ([0, 1, 0], [1, 0, 0]),
 ([1, 0, 0], [0, 0, 1]),
 ([1, 0, 0], [0, 1, 0])]

Nonetheless, it can happen that different couples of codes generate the same pattern. In that sense, the function `get_unique_codes` generates codes resulting in unique patterns. It should be the generic one:

In [10]:
pf.get_unique_codes(7)

[([], [0, 0, 0]),
 ([0, 0, 1], [0, 1, 0]),
 ([0, 0, 1], [1, 0, 0]),
 ([0, 1, 0], [0, 0, 1]),
 ([0, 1, 0], [1, 0, 0]),
 ([1, 0, 0], [0, 0, 1]),
 ([1, 0, 0], [0, 1, 0])]

### Switching between pattern of ones and indexed pattern

It can be useful to switch from an indexed pattern to a pattern of ones, or vice-versa. This can be made by using the functions presented below:

In [11]:
pattern_of_ones = pf.make_polytope_pattern(dimension = 4, adding_code = [], deleting_code = [])
print("Pattern of ones:\n{}\n".format(pattern_of_ones))

indexed_pattern = pf.index_this_pattern(pattern_of_ones)
print("Previous pattern, indexed:\n{}\n".format(indexed_pattern))

re_pattern_of_ones = pf.extract_pattern_from_indexed_pattern(indexed_pattern)
print("Going back to pattern of ones pattern, indexed:\n{}".format(re_pattern_of_ones))

Pattern of ones:
[[[[1, 1], [1, 1]], [[1, 1], [1, 1]]], [[[1, 1], [1, 1]], [[1, 1], [1, 1]]]]

Previous pattern, indexed:
[[[[0, 1], [2, 3]], [[4, 5], [6, 7]]], [[[8, 9], [10, 11]], [[12, 13], [14, 15]]]]

Going back to pattern of ones pattern, indexed:
[[[[1, 1], [1, 1]], [[1, 1], [1, 1]]], [[[1, 1], [1, 1]], [[1, 1], [1, 1]]]]


The following function indicates whether a pattern is indexed or not (*i.e.* is a pattern of ones):

In [12]:
pattern_of_ones = pf.make_polytope_pattern(dimension = 4, adding_code = [], deleting_code = [])
boolean = pf.is_indexed_pattern(pattern_of_ones)
print("Is {} an indexed pattern?\n{}\n".format(pattern_of_ones, boolean))

indexed_pattern = pf.index_this_pattern(pattern_of_ones)
boolean = pf.is_indexed_pattern(indexed_pattern)
print("Is {} an indexed pattern?\n{}".format(indexed_pattern, boolean))

Is [[[[1, 1], [1, 1]], [[1, 1], [1, 1]]], [[[1, 1], [1, 1]], [[1, 1], [1, 1]]]] an indexed pattern?
False

Is [[[[0, 1], [2, 3]], [[4, 5], [6, 7]]], [[[8, 9], [10, 11]], [[12, 13], [14, 15]]]] an indexed pattern?
True


## Get informations from a pattern
We've previously seen how to construct a pattern. In practice, one also needs to access to informations about a particular pattern.

Let's consider a particular pattern as an example:

In [13]:
pattern = pf.make_indexed_pattern(dimension = 4, adding_code = [1,0,0,0], deleting_code = [0,1,0,1])
pattern

[[[[0, 1], [2, 3]], [[4, 5], [6, (7, 8)]]], [[[9, 10]], [[11, 12]]]]

Two important informations about a pattern are its dimension and its size. Both can be accessed with `get_pattern_dimension` and `get_pattern_size` functions respectively:

In [14]:
pf.get_pattern_dimension(pattern)

4

In [15]:
pf.get_pattern_size(pattern)

13

## Flattening patterns
Two functions exist in order to turn a pattern (nested lists) into a simple list. We call this operation "flattening".

The first one flattens everything in the pattern, including the tuples (which indicate addition):

In [16]:
pf.flatten_pattern(pattern)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

The second one keeps tuples as tuples, but flatten the nesting of list:

In [17]:
pf.flatten_nested_list(pattern)

[0, 1, 2, 3, 4, 5, 6, (7, 8), 9, 10, 11, 12]

This function may be used in order to keep track of the additions.

## Applying chords on a pattern
Finally, as polytopes are meant to represent musical elements, we can apply a segment/list of chords on a pattern. Note though that this function is developped for **visualization purpose only**, and is not adapted for cost or complexity computation.

In [18]:
segment = ['Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'Eb', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ab', 'Ebm', 'Ebm', 'Ebm', 'Ebm']
print("List of chords:\n{}\n".format(segment))

size = len(segment)
adding_code, deleting_code = pf.get_unique_codes(size)[0]
import math
dimension = round(math.log(size,2))
#pattern = pf.make_polytope_pattern(dimension = dimension, adding_code = adding_code, deleting_code = deleting_code)
pattern = pf.make_indexed_pattern(dimension = dimension, adding_code = adding_code, deleting_code = deleting_code)
print("The pattern:\n{}\n".format(pattern))

chords_on_pattern = pf.apply_chords_on_pattern(pattern, segment)
print("The pattern, with chords as elements:\n{}\n".format(chords_on_pattern))

List of chords:
['Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'Eb', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ab', 'Ebm', 'Ebm', 'Ebm', 'Ebm']

The pattern:
[[[[[0, 1], [2, 3]], [[4, 5], [6, 7]]], [[[8, 9], [10, 11]], [[12, 13], [14, 15]]]], [[[[16, 17], [18, 19]], [[20, 21], [22, 23]]], [[[24, 25], [26, 27]], [[(28, 29), (30, 31)], [(32, 33), (34, 35)]]]]]

The pattern, with chords as elements:
[[[[['Eb', 'Eb'], ['Eb', 'Eb']], [['Abm', 'Abm'], ['Abm', 'Abm']]], [[['E', 'E'], ['E', 'E']], [['F#', 'F#'], ['Eb', 'Eb']]]], [[[['Abm', 'Abm'], ['Abm', 'Abm']], [['Abm', 'Abm'], ['E', 'Eb']]], [[['Ebm', 'Ebm'], ['Ebm', 'Ebm']], [[('Ebm', 'Ebm'), ('Ebm', 'Ab')], [('Ebm', 'Ebm'), ('Ebm', 'Ebm')]]]]]



# Indexing elements, antecedents and successors
In this part, we will focus on linking and accessing elements in the pattern. More particularly, we will focus on the paradigm developed by C. Guichaoua [1]. In this paradigm, elements are studied in comparison with the previous ones. In that sense, we need to know the position of each element, and to be able to compare this position with others.

**Disclaimer: as we need to access to particular elements, all patterns will have to be indexed ones.**

All the code related to this part is contained in the file pattern_manip.py.

In [19]:
from polytopes import pattern_manip as pm

## Index of an element
We developped a system of indexation for every element, based on dichotomy. To find an element, we will recursively indicate whether the element is in the first (left) or the second (right) nested polytope, and search deeper into it. To be precise, we will index each element with a list of booleans, where:
 - 0 indicates that this element is on the 1st (left) nested polytope,
 - 1 indicates that this element is on the 2nd (right) nested polytope.

With recursion, we can find every element with as much booleans as the dimension of the polytope.

<img src="imgs/index_gif.gif" width="700"/>

A special case appears for additions in the polytope, because we need to specify which element on the added edge we are studying. Similarly to addition in patterns, we use tuples to indicate which element we are looking at. To remain consistent with the dichotomy principle, the tuple contains two booleans, the first one indicating the position of the edge in the last dimension (as for any other element), and the second representing the position of the element on the edge (0 for left, *i.e.* the original element, and 1 for the right, *i.e.* the added element).

Using a tuple has the advantage of specifying that we're dealing with an addition, and not another dimension. Furthermore, indexes still contains $d$ elements (with $d$ the dimension), with the last element being a tuple with two booleans rather than a unique boolean.

<img src="imgs/indexes_examples.png" width="700"/>

In both cases, one should use function ``get_index_from_element``, as shown below:

In [20]:
pattern = pf.make_indexed_pattern(dimension = 3, adding_code = [0,1,0], deleting_code = [])
print("Pattern: {} (the same as the right one on the previous figure)".format(pattern))

ind_three = pm.get_index_from_element(3, pattern)
print("a_3 has index: {}".format(ind_three))

ind_five = pm.get_index_from_element(5, pattern)
print("a_5 has index: {}".format(ind_five))

ind_six = pm.get_index_from_element(6, pattern)
print("a_6 has index: {}".format(ind_six))

Pattern: [[[0, 1], [2, 3]], [[4, (5, 6)], [7, (8, 9)]]] (the same as the right one on the previous figure)
a_3 has index: [0, 1, 1]
a_5 has index: [1, 0, (1, 0)]
a_6 has index: [1, 0, (1, 1)]


The inverse operation also exists, with the function ``get_element_from_index``:

In [21]:
pattern = pf.make_indexed_pattern(dimension = 3, adding_code = [0,1,0], deleting_code = [])
print("Pattern: {} (the same as before)".format(pattern))

element = pm.get_element_from_index([0,1,1], pattern)
print("[0,1,1] is the element: {}".format(element))

element = pm.get_element_from_index([1, 0, (1, 0)], pattern)
print("[1, 0, (1, 0)] is the element: {}".format(element))

element = pm.get_element_from_index([1, 0, (1, 1)], pattern)
print("[1, 0, (1, 0)] is the element: {}".format(element))

element = pm.get_element_from_index([0, 0, (1, 1)], pattern)
print("[0, 0, (1, 0)] is the element: {} (should be None, as there is no such element added)".format(element))

Pattern: [[[0, 1], [2, 3]], [[4, (5, 6)], [7, (8, 9)]]] (the same as before)
[0,1,1] is the element: 3
[1, 0, (1, 0)] is the element: 5
[1, 0, (1, 0)] is the element: 6
[0, 0, (1, 0)] is the element: None (should be None, as there is no such element added)


## Antecedents and successors
A new concept, defined in [1], is the **antecedent** of an element.

Looking at a polytope as an oriented graph, edges become oriented arrows, oriented in chronological order ($a_0$ &rarr; $a_1$). In this example, $a_0$ is spawning an arrow pointing towards $a_1$. $a_0$ is called "antecedent" of $a_1$, and $a_1$ is called "successor" of $a_0$.

Hence, the antecedents of an element are all elements spawning an arrow pointing towards them, and the successors of an element are the tip of all arrows they spawn.

For instance, in the following polytope, $a_3$ has two antecedents ($a_1$ and $a_2$), $a_4$ has one ($a_3$) and $a_5$ has one ($a_0$).

<img src="imgs/dim_3_add_100_del_001.png" width="200"/>

> NB: the definition of antecedents in [1] is wider, including every element originating an oriented *path* to the current one. In that definition, $a_0$ would be antecedent of every element. We didn't followed this definition and restricted ourselves with "direct" antecedents, *i.e.* element spawning a direct arrow with the current element.

Antecedents of an elements can be computed using the functions ``get_antecedents_from_element`` or ``get_antecedents_from_index`` depending or whether the studied element is in the form of an element (element 1, 2, 3, etc) or as an index.

In [22]:
pattern = pf.make_indexed_pattern(dimension = 3, adding_code = [1,0,0], deleting_code = [0,0,1])

ant_elt = pm.get_antecedents_from_element(3, pattern)
print("Antecedents of the element 3: {}".format(ant_elt))

ant_idx = pm.get_antecedents_from_index([0,1,1], pattern)
print("Antecedents of the element [0,1,1]: {}".format(ant_idx))

Antecedents of the element 3: [1, 2]
Antecedents of the element [0,1,1]: [1, 2]


One can also retreive the index of the antecedent from the index of the element, with the function ``get_antecedents_index_from_index``:

In [23]:
pm.get_antecedents_index_from_index([0,1,1])
# This function does not need the pattern, but some of its outputs may not exist in every pattern

[[0, 0, 1], [0, 1, 0]]

In the paradigm of [1], each antecedent is associated with a "pivot" element, in order to construct a square system (implication system of 4 elements) with the primer ($a_0$, the first element of the polytope). In our example above, if we consider the antecedent $a_5$ of $a_6$, its pivot is $a_1$, because it defines a square system $(a_0, a_1, a_5, a_6)$. For further details, the reader should refer to [1] or [TODO: Add My Own Reference].

In the code, this is made with the function ``get_pivot_from_index`` (NB: the antecedent needs to be under its index form, and it returns the pivot as element, not index):

In [24]:
pattern = pf.make_indexed_pattern(dimension = 3, adding_code = [1,0,0], deleting_code = [0,0,1])

element = 6
elt_idx = pm.get_index_from_element(element, pattern)
ant_idx = pm.get_antecedents_index_from_index(elt_idx)[0]
pivot = pm.get_pivot_from_index(elt_idx, ant_idx, pattern)
print("Element: {}, antecedent: {}, pivot: {}".format(element, pm.get_element_from_index(ant_idx, pattern), pivot))

Element: 6, antecedent: 1, pivot: 5


Finally, the function ``get_antecedents_with_pivots_from_index`` returns couples (as tuples) of antecedents and their pivot from the index of an element:

In [25]:
pm.get_antecedents_with_pivots_from_index(elt_idx, pattern)

[(1, 5), (5, 1)]

The same functions exist for successors of an element, *i.e.* ``get_successors_from_element``, ``get_successors_from_index``, which returns the successors of an element (under its element form) when, respectively, it's an element and as an index, and the function `get_successors_index_from_index`, which returns the successors of an element under its index form, from the element as an index.

For example, to find the successors of $a_1$ (which are $a_3$ and $a_6$), one can use the functions:

In [26]:
pattern = pf.make_indexed_pattern(dimension = 3, adding_code = [1,0,0], deleting_code = [0,0,1])

element = 1
suc_elt_from_elt = pm.get_successors_from_element(element, pattern)
print("Successors of {}: {}".format(element, suc_elt_from_elt))

elt_idx = pm.get_index_from_element(element, pattern)
suc_elt_from_idx = pm.get_successors_from_index(elt_idx, pattern)
print("Successors of {}: {}".format(elt_idx, suc_elt_from_idx))

suc_idx_from_idx = pm.get_successors_index_from_index(elt_idx)
print("Successors of {}: {}".format(elt_idx, suc_idx_from_idx))

Successors of 1: [6, 3]
Successors of [0, 0, 1]: [6, 3]
Successors of [0, 0, 1]: [[1, 0, 1], [0, 1, 1]]


# PPP
An interesting paradigm developed in [2] by C. Louboutin is the PPP, for Primer Preserving Permutation. The idea is to look at relation in a non-sequential way, and try to find implication systems which explain a music sequence in a different logic, and with different functions between elements (for example, looking at the first beats of all bars between them, then second beats between them, etc).

In this seminal work, PPP were only defined for regular polytopes, *i.e.* polytopes without alteration, with $2^n$ element ($n$ being the dimension). An illustration example is shown below:

<img src="imgs/ppp_16.png" width="400"/>

This work has been extended to irregular polytopes, and can be used with the function `generate_ppp`, starting from an indexed pattern.

> This extension is not explained here, but is in [TODO: Add My Own Reference]. In a nutshell, the idea is to define the interesting faces with different couples of edges (seen as vectors), and interpret the polytope through these faces. In that sense, alteration does not change the paradigm, and are permuted too.

An example of PPP on an irregular polytope is presented in the figure below:

<img src="imgs/irregular_ppp.png" width="800"/>

The code is presented below:

In [27]:
pattern = pf.make_indexed_pattern(dimension = 4, adding_code = [1,0,0,1], deleting_code = [0,0,0,1])
print("Pattern: {}\n".format(pattern))

all_ppps = pm.generate_ppp(pattern)

for idx, a_ppp in enumerate(all_ppps):
    print("PPP {} of this pattern:\n{}\n".format(idx, a_ppp))

Pattern: [[[[0, 1], [2, 3]], [[4, 5], [(6, 7), (8, 9)]]], [[[10, 11], [12, 13]], [[14, 15]]]]

PPP 0 of this pattern:
[[[[0, 1], [2, 3]], [[4, 5], [(6, 7), (8, 9)]]], [[[10, 11], [12, 13]], [[14, 15]]]]

PPP 1 of this pattern:
[[[[0, 1], [4, 5]], [[2, 3], [(6, 7), (8, 9)]]], [[[10, 11], [14, 15]], [[12, 13]]]]

PPP 2 of this pattern:
[[[[0, 1], [10, 11]], [[2, 3], [12, 13]]], [[[4, 5], [14, 15]], [[(6, 7), (8, 9)]]]]

PPP 3 of this pattern:
[[[[0, 2], [4, (6, 7)]], [[1, 3], [5, (8, 9)]]], [[[10, 12], [14]], [[11, 13], [15]]]]

PPP 4 of this pattern:
[[[[0, 2], [10, 12]], [[1, 3], [11, 13]]], [[[4, (6, 7)], [14]], [[5, (8, 9)], [15]]]]

PPP 5 of this pattern:
[[[[0, 4], [10, 14]], [[1, 5], [11, 15]]], [[[2, (6, 7)], [12]], [[3, (8, 9)], [13]]]]



# Polytopical costs
The goal of this paradigm is to compute costs for musical sequences. This costs are then fed to dynamic programming algorithm, computing a segmentation of the whole music piece.

Costs are explained in [TODO: Add Own Reference].

One important notion in these costs is the relation between the musical elements. Indeed, as an edge links two musical elements, it should model a relation between them. Hence, a relation space between the musical elements must be defined.

In addition, in both paradigms [1] and [2], some elements need to be anticipated from the context. Concretley, this means that some relations of the past of an element are composed to form a new "virutal" (anticipated) element. Hence, the aformentioned relation space needs to be ***stable par composition*** (need help here, I only found "internal binary operation" in english to say what I want, is this correct?).

Finally, the relation space needs to be commutative, to simplify the model of implication.

In this work, we chose to represent musical elements as major or minor chords, and relation between them in the "circle of triads". This circle represents every major and minor chords, ordered following the circle of fifths. Minor and major chords are alternated, following the order of relative fifths. It is presented below (from [1]):

<img src="imgs/triad_circle.png" width="400"/>

Hence, a relation between two elements wil be the signed rotation (in number of steps) between these two chords.

It is to be noted that this relation space is limited, and more work should be made on this topic to accurately evaluate the potential of polytopes.

The file containing the code for computing the costs is the file `polytopical_costs.py`.

In [28]:
from polytopes import polytopical_costs as pc

## Cost defined by C. Guichaoua [1]
The first cost is the one defined by C. Guichaoua in [1], also explained in [TODO: Add Own reference]. In a nutshell, in consists in anticipating elements with their antecedents, and increase the cost by 1 if the element is different than the anticipated one.

To find the best polytope according to this cost, one can use the function `best_guichaoua_cost_segment`, returning the optimal polytope according to this cost function, along with the resulting cost.

In [29]:
segment = ['Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'Eb', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ab', 'Ebm', 'Ebm', 'Ebm', 'Ebm']
print("List of chords:\n{}\n".format(segment))

cost, pattern = pc.best_guichaoua_cost_segment(segment)
print("Optimal pattern:\n{}\nResulting in cost: {}".format(pattern, cost))

chords_on_pattern = pf.apply_chords_on_pattern(pattern, segment)
print("The optimal pattern, with chords as elements:\n{}\n".format(chords_on_pattern))

List of chords:
['Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'Eb', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ab', 'Ebm', 'Ebm', 'Ebm', 'Ebm']

Optimal pattern:
[[[[[0, 1], [2, 3]], [[4, 5], [6, 7]]], [[[8, 9], [10, 11]], [[12, 13], [14, 15]]]], [[[[16, 17], [18, 19]], [[20, 21], [22, 23]]], [[[(24, 25), (26, 27)], [(28, 29), (30, 31)]], [[(32, 33), (34, 35)]]]]]
Resulting in cost: 12
The optimal pattern, with chords as elements:
[[[[['Eb', 'Eb'], ['Eb', 'Eb']], [['Abm', 'Abm'], ['Abm', 'Abm']]], [[['E', 'E'], ['E', 'E']], [['F#', 'F#'], ['Eb', 'Eb']]]], [[[['Abm', 'Abm'], ['Abm', 'Abm']], [['Abm', 'Abm'], ['E', 'Eb']]], [[[('Ebm', 'Ebm'), ('Ebm', 'Ebm')], [('Ebm', 'Ebm'), ('Ebm', 'Ab')]], [[('Ebm', 'Ebm'), ('Ebm', 'Ebm')]]]]]



NB: this function can also compute the score with penalties, but they are not presented here. You should refer to [1] and the documentation of this function in order to use them.

**Note pour moi-même: check le code.**

## Cost of C. Louboutin [2], extended to irregular polytopes
A second cost is the one used in C. Louboutin's paradigm, presented in [2]. The idea of his model is to increase the score according to the "harmonic" distance between elements (*i.e.* the distance between chords in the circle of triads). His cost is based on the System and Contrast model, and each cost depends on a PPP.

As presented before, PPP have been expanded to irregular polytopes, so this cost have been expanded to all irregular polytopes. This new cost, called "Louboutin cost", is presented in [TODO: Add Own reference].

To find the best polytope according to this cost, one can use the function `best_louboutin_cost_segment`, returning the optimal polytope according to this cost function, along with the resulting cost.

In [30]:
segment = ['Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'Eb', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ab', 'Ebm', 'Ebm', 'Ebm', 'Ebm']
print("List of chords:\n{}\n".format(segment))

cost, pattern = pc.best_louboutin_cost_segment(segment)
print("Optimal pattern:\n{}\nResulting in cost: {}\n".format(pattern, cost))

chords_on_pattern = pf.apply_chords_on_pattern(pattern, segment)
print("The optimal pattern, with chords as elements:\n{}\n".format(chords_on_pattern))

List of chords:
['Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'Eb', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ab', 'Ebm', 'Ebm', 'Ebm', 'Ebm']

Optimal pattern:
[[[[[0, 1], [2, 3]], [[4, 5], [6, 7]]], [[[8, 9], [10, 11]], [[12, 13], [14, 15]]]], [[[[16, 17], [18, 19]], [[20, 21], [22, 23]]], [[[24, 25], [26, 27]], [[(28, 29), (30, 31)], [(32, 33), (34, 35)]]]]]
Resulting in cost: 63

The optimal pattern, with chords as elements:
[[[[['Eb', 'Eb'], ['Eb', 'Eb']], [['Abm', 'Abm'], ['Abm', 'Abm']]], [[['E', 'E'], ['E', 'E']], [['F#', 'F#'], ['Eb', 'Eb']]]], [[[['Abm', 'Abm'], ['Abm', 'Abm']], [['Abm', 'Abm'], ['E', 'Eb']]], [[['Ebm', 'Ebm'], ['Ebm', 'Ebm']], [[('Ebm', 'Ebm'), ('Ebm', 'Ab')], [('Ebm', 'Ebm'), ('Ebm', 'Ebm')]]]]]



**Note pour moi-même: check le code (surtout celui-là).**

# Dynamic minimization
Finally, to compute the dynamic minimization on a song, one should use the module `segmentation_algorithms.py`.

In [31]:
from polytopes import segmentation_algorithms as sa

Functions corresponding to the previous models are respectively `dynamic_minimization_guichaoua` and `dynamic_minimization_louboutin`.

NB: a good practice would be to have only one dynamic minimization function, with costs specified as an argument. This should be made in future versions.

In [32]:
# 001.manual.simple.seq, from RWC POP
song = ['Abm', 'Abm', 'Abm', 'Abm', 'F#', 'F#', 'F#', 'F#', 'E', 'E', 'E', 'E', 'F#', 'F#', 'F#', 'F#', 'F#', 'F#', 'F#', 'F#', 'Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'B', 'B', 'B', 'B', 'Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'F#', 'F#', 'F#', 'F#', 'C#', 'C#', 'C#', 'C#', 'E', 'E', 'F#', 'F#', 'Abm', 'Abm', 'Abm', 'Abm', 'F#', 'F#', 'F#', 'F#', 'C#', 'C#', 'C#', 'C#', 'E', 'E', 'F#', 'F#', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'F', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Bb', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'F', 'Ab', 'Ab', 'Bb', 'Bb', 'Cm', 'Cm', 'Cm', 'Cm', 'Ab', 'Ab', 'Ab', 'Ab', 'Eb', 'Eb', 'Eb', 'Eb', 'G', 'G', 'G', 'G', 'Cm', 'Cm', 'Cm', 'Cm', 'Ab', 'Ab', 'Ab', 'Ab', 'Eb', 'Eb', 'Eb', 'Eb', 'G', 'G', 'G', 'G', 'Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'B', 'B', 'B', 'B', 'Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'F#', 'F#', 'F#', 'F#', 'C#', 'C#', 'C#', 'C#', 'E', 'E', 'F#', 'F#', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'F', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Bb', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'Cm', 'F', 'Ab', 'Ab', 'Bb', 'Bb', 'Cm', 'Cm', 'Cm', 'Cm', 'Ab', 'Ab', 'Ab', 'Ab', 'Eb', 'Eb', 'Eb', 'Eb', 'G', 'G', 'G', 'G', 'Cm', 'Cm', 'Cm', 'Cm', 'Ab', 'Ab', 'Ab', 'Ab', 'Eb', 'Eb', 'Eb', 'Eb', 'G', 'G', 'G', 'G', 'Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'B', 'B', 'B', 'B', 'Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'Eb', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ab', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'C#', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ebm', 'Ab', 'C#', 'C#', 'F#', 'F#', 'B', 'B', 'E', 'E', 'A', 'A', 'D', 'D', 'Eb', 'Eb', 'Eb', 'Eb', 'Eb', 'Eb', 'Eb', 'Eb', 'Ab', 'Ab', 'Ab', 'Ab', 'Eb', 'Eb', 'Eb', 'Eb', 'G', 'G', 'G', 'G', 'Cm', 'Cm', 'Cm', 'Cm', 'Ab', 'Ab', 'Ab', 'Ab', 'Eb', 'Eb', 'Eb', 'Eb', 'G', 'G', 'G', 'G', 'Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'B', 'B', 'B', 'B', 'Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'B', 'B', 'B', 'B', 'Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'F#', 'F#', 'F#', 'F#', 'C#', 'C#', 'C#', 'C#', 'E', 'E', 'F#', 'F#', 'Abm', 'Abm', 'Abm', 'Abm', 'F#', 'F#', 'F#', 'F#', 'C#', 'C#', 'C#', 'C#', 'E', 'E', 'F#', 'F#', 'F#', 'F#', 'Abm', 'Abm', 'Abm', 'Abm']

In [33]:
# Guichaoua
frontiers, cost = sa.dynamic_minimization_guichaoua(song)
print("Frontiers of this algorithm:\n{}\nWith cost: {}".format(frontiers, cost))

c:\users\amarmore\desktop\projects\phd main projects\on git\code\polytopes and relation\polytopes\segmentation_algorithms.py:487: UserWarning: No Polytope is available for this size of segment. Trying sequential cost instead.
  warnings.warn("No Polytope is available for this size of segment. Trying sequential cost instead.")


Frontiers of this algorithm:
[0, 20, 52, 84, 112, 144, 176, 192, 220, 252, 284, 292, 332, 364, 396, 428, 462]
With cost:113.25


In [34]:
# Louboutin
frontiers, cost = sa.dynamic_minimization_louboutin(song)
print("Frontiers of this algorithm:\n{}\nWith cost: {}".format(frontiers, cost))

c:\users\amarmore\desktop\projects\phd main projects\on git\code\polytopes and relation\polytopes\segmentation_algorithms.py:616: UserWarning: No Polytope is available for this size of segment. Trying sequential cost instead.
  warnings.warn("No Polytope is available for this size of segment. Trying sequential cost instead.")


KeyboardInterrupt: 